In [1]:
from langchain.chat_models import ChatVertexAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import HumanMessage, SystemMessage

In [2]:
chat=ChatVertexAI()

cloudpickle is not installed. Please call `pip install google-cloud-aiplatform[preview]`.


In [3]:
messages = [
    SystemMessage(
        content="You are a helpful assistant that translates English to French."
    ),
    HumanMessage(
        content="Translate this sentence from English to French. I love programming."
    ),
]
chat(messages)

AIMessage(content=" J'aime la programmation.", additional_kwargs={}, example=False)

# Prompt Template

In [4]:
from langchain import PromptTemplate

prompt_template = PromptTemplate.from_template(
    "Tell me a {adjective} joke about {content}."
)
prompt_template.format(adjective="gruesome", content="goats")

'Tell me a gruesome joke about goats.'

# Chat Template

In [5]:
from langchain.prompts import ChatPromptTemplate

template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI bot. Your name is {name}."),
    ("human", "Hello, how are you doing?"),
    ("ai", "I'm doing well, thanks!"),
    ("human", "{user_input}"),
])

messages = template.format_messages(
    name="Bob",
    user_input="What is your name?"
)

print(messages)

ValidationError: 8 validation errors for ChatPromptTemplate
messages -> 0
  value is not a valid dict (type=type_error.dict)
messages -> 0
  value is not a valid dict (type=type_error.dict)
messages -> 1
  value is not a valid dict (type=type_error.dict)
messages -> 1
  value is not a valid dict (type=type_error.dict)
messages -> 2
  value is not a valid dict (type=type_error.dict)
messages -> 2
  value is not a valid dict (type=type_error.dict)
messages -> 3
  value is not a valid dict (type=type_error.dict)
messages -> 3
  value is not a valid dict (type=type_error.dict)

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.chat import SystemMessage, HumanMessagePromptTemplate

template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(
                "You are a helpful assistant that re-writes the user's text to "
                "sound more upbeat."
            )
        ),
        HumanMessagePromptTemplate.from_template("{text}"),
    ]
)

from langchain.chat_models import ChatVertexAI

llm = ChatVertexAI()
llm(template.format_messages(text='i dont like eating tasty things.'))

AIMessage(content=' You have a very refined palate and prefer to eat healthy and nutritious foods.', additional_kwargs={}, example=False)

# Creating a Chain

In [ ]:
import os
from getpass import getpass
from langchain import PromptTemplate

prompt_template = PromptTemplate.from_template(
    "Tell me a {adjective} joke about {topic}"
)

prompt_template.format(adjective="offensive", topic="racism")

'Tell me a offensive joke about racism'

# Declaring all the models here

In [ ]:
from langchain.embeddings import VertexAIEmbeddings
from pydantic import BaseModel
# Utility functions for Embeddings API with rate limiting
def rate_limit(max_per_minute):
    period = 60 / max_per_minute
    print("Waiting")
    while True:
        before = time.time()
        yield
        after = time.time()
        elapsed = after - before
        sleep_time = max(0, period - elapsed)
        if sleep_time > 0:
            print(".", end="")
            time.sleep(sleep_time)


class CustomVertexAIEmbeddings(VertexAIEmbeddings, BaseModel):
    requests_per_minute: int
    num_instances_per_batch: int

    # Overriding embed_documents method
    def embed_documents(self, texts: List[str]):
        limiter = rate_limit(self.requests_per_minute)
        results = []
        docs = list(texts)

        while docs:
            # Working in batches because the API accepts maximum 5
            # documents per request to get embeddings
            head, docs = (
                docs[: self.num_instances_per_batch],
                docs[self.num_instances_per_batch :],
            )
            chunk = self.client.get_embeddings(head)
            results.extend(chunk)
            next(limiter)

        return [r.values for r in results]

TypeError: metaclass conflict: the metaclass of a derived class must be a (non-strict) subclass of the metaclasses of all its bases

In [ ]:
from langchain.llms import VertexAI

# LLM model
llm = VertexAI(
    model_name="text-bison-32k",
    max_output_tokens=256,
    temperature=0.1,
    top_p=0.8,
    top_k=40,
    verbose=True,
)

# Chat
chat = ChatVertexAI(
    model_name = "chat-bison-32k",
)

# Embedding
EMBEDDING_QPM = 100
EMBEDDING_NUM_BATCH = 5
embeddings = CustomVertexAIEmbeddings(
    requests_per_minute=EMBEDDING_QPM,
    num_instances_per_batch=EMBEDDING_NUM_BATCH,
)

NameError: name 'CustomVertexAIEmbeddings' is not defined

In [ ]:
# token counting
import asyncio

from langchain.callbacks import get_openai_callback
from langchain.llms import OpenAI

llm = OpenAI(temperature=0)
with get_openai_callback() as cb:
    llm("What is the square root of 4?")

total_tokens = cb.total_tokens
assert total_tokens > 0

with get_openai_callback() as cb:
    llm("What is the square root of 4?")
    llm("What is the square root of 4?")

assert cb.total_tokens == total_tokens * 2

# You can kick off concurrent runs from within the context manager
with get_openai_callback() as cb:
    await asyncio.gather(
        *[llm.agenerate(["What is the square root of 4?"]) for _ in range(3)]
    )

assert cb.total_tokens == total_tokens * 3

# The context manager is concurrency safe
task = asyncio.create_task(llm.agenerate(["What is the square root of 4?"]))
with get_openai_callback() as cb:
    await llm.agenerate(["What is the square root of 4?"])

await task
assert cb.total_tokens == total_tokens

In [ ]:
from langchain.callbacks import StdOutCallbackHandler
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate

handler = StdOutCallbackHandler()
prompt = PromptTemplate.from_template("1 + {number} = ")

# Constructor callback: First, let's explicitly set the StdOutCallbackHandler when initializing our chain
chain = LLMChain(llm=llm, prompt=prompt, callbacks=[handler])
chain.run(number=2)

# Use verbose flag: Then, let's use the `verbose` flag to achieve the same result
chain = LLMChain(llm=llm, prompt=prompt, verbose=True)
chain.run(number=2)

# Request callbacks: Finally, let's use the request `callbacks` to achieve the same result
chain = LLMChain(llm=llm, prompt=prompt)
chain.run(number=2, callbacks=[handler])



> Entering new LLMChain chain...
Prompt after formatting:
1 + 2 = 

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
1 + 2 = 

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
1 + 2 = 

> Finished chain.


' 1 + 2 = 3'

# Retrival

In [ ]:
# Document loaders
from langchain.document_loaders import TextLoader

loader = TextLoader("./index.md")
loader.load()

[Document(page_content='hello\n\n# What is your name?\n', metadata={'source': './index.md'})]

# Web based loader

In [ ]:

from langchain.document_loaders import WebBaseLoader
     

loader = WebBaseLoader("https://cloud.google.com/blog/products/ai-machine-learning/generative-ai-applications-with-vertex-ai-palm-2-models-and-langchain")     
data = loader.load()

In [ ]:
print(f"Found {len(data)} comments")
print(f"Here's a sample:\n\n{''.join([x.page_content[:500] for x in data[:2]])}")

Found 1 comments
Here's a sample:

Generative AI applications with Vertex AI PaLM 2 Models and LangChain | Google Cloud BlogJump to ContentCloudBlogContact sales Get started for free CloudBlogSolutions & technologyAI & Machine LearningAPI ManagementApplication DevelopmentApplication ModernizationChrome EnterpriseComputeContainers & KubernetesData AnalyticsDatabasesDevOps & SREMaps & GeospatialSecurity & IdentityInfrastructureInfrastructure ModernizationNetworkingProductivity & CollaborationSAP on Google CloudStorage & Data Transf


# Memory

In [ ]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

conversation = ConversationChain(
    llm=llm, verbose=True, memory=ConversationBufferMemory()
)

conversation.predict(input="Hi there!")
     

conversation.predict(input="What is the capital of Spain?")
     

conversation.predict(input="What are some popular places I can see in France?")
     

conversation.predict(input="What question did I ask first?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi there!
AI:

> Finished chain.


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there!
AI:  Hi there! It's nice to meet you. How can I help you today?
Human: What is the capital of Spain?
AI:

> Finished chain.


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lo

' You asked what the capital of Spain is.'

# Chains

### Sequential Chain

In [ ]:
from langchain.chains import LLMChain, SimpleSequentialChain
from langchain.prompts import PromptTemplate

In [ ]:
template = """Your job is to come up with a classic dish from the area that the users suggests.
% USER LOCATION
{user_location}

YOUR RESPONSE:
"""
prompt_template = PromptTemplate(input_variables=["user_location"], template=template)

# Holds my 'location' chain
location_chain = LLMChain(llm=llm, prompt=prompt_template)

In [ ]:
template = """Given a meal, give a short and simple recipe on how to make that dish at home.
% MEAL
{user_meal}

YOUR RESPONSE:
"""
prompt_template = PromptTemplate(input_variables=["user_meal"], template=template)

# Holds my 'meal' chain
meal_chain = LLMChain(llm=llm, prompt=prompt_template)

In [ ]:
overall_chain = SimpleSequentialChain(chains=[location_chain, meal_chain], verbose=True)

review = overall_chain.run("Mumbai")



> Entering new SimpleSequentialChain chain...
A classic dish from Mumbai is Pav Bhaji, a spicy vegetable curry served with a soft bread roll.
Pav Bhaji Recipe:

Ingredients:
- 2 tablespoons vegetable oil
- 1 teaspoon cumin seeds
- 1 onion, finely chopped
- 2 cloves garlic, minced
- 1 teaspoon ginger, minced
- 1 green chili, finely chopped
- 1 teaspoon turmeric powder
- 1 teaspoon coriander powder
- 1 teaspoon garam masala
- 2 large potatoes, peeled and diced
- 1 large carrot, peeled and diced
- 1 cup green peas
- 1 cup cauliflower florets
- 1 cup tomato puree
- Salt to taste
- 2 tablespoons butter
- Pav buns

Instructions:
1. Heat the oil in a large pan over medium heat.
2. Add the cumin seeds and let them sizzle for a few seconds.
3. Add the onion, garlic, ginger, and green chili and sauté until the onion is softened.
4. Add the turmeric, coriander, and garam masala and stir to combine.
5. Add the potatoes, carrot, green peas, and cauliflower and stir to combine.
6. Add the tomato p

### Summarisation Chain

In [ ]:
from langchain.chains.summarize import load_summarize_chain
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = WebBaseLoader(
    "https://cloud.google.com/blog/products/ai-machine-learning/how-to-use-grounding-for-your-llms-with-text-embeddings"
)
documents = loader.load()

print(f"# of words in the document = {len(documents[0].page_content)}")

# Get your splitter ready
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=50)

# Split your docs into texts
texts = text_splitter.split_documents(documents)

# There is a lot of complexity hidden in this one line. I encourage you to check out the video above for more detail
chain = load_summarize_chain(llm, chain_type="map_reduce", verbose=True)
chain.run(texts)

# of words in the document = 13656


> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"How to use Grounding for your LLMs with text embeddings | Google Cloud BlogJump to ContentCloudBlogContact sales Get started for free CloudBlogSolutions & technologyAI & Machine LearningAPI ManagementApplication DevelopmentApplication ModernizationChrome EnterpriseComputeContainers & KubernetesData AnalyticsDatabasesDevOps & SREMaps & GeospatialSecurity & IdentityInfrastructureInfrastructure ModernizationNetworkingProductivity & CollaborationSAP on Google CloudStorage & Data TransferSustainabilityEcosystemIT LeadersIndustriesFinancial ServicesHealthcare & Life SciencesManufacturingMedia & EntertainmentPublic SectorRetailSupply ChainTelecommunicationsPartnersStartups & SMBTraining & CertificationsInside Google CloudGoogle Cloud Next & EventsGoogle Maps PlatformGoogle WorkspaceDevelopers & Practitione

" This article explains how to use Grounding for LLMs with text embeddings to integrate LLMs and AI chatbots with existing IT systems, databases, and business data. It covers how to handle the hallucination issues in AI chatbots in order to build a reliable service. It introduces Google Cloud's Vertex AI Model Garden, Nomic AI's Embeddings API, and Google Research's ScaNN algorithm as popular tools for grounding LLMs. It also provides resources for developers and practitioners to get started with AI and Machine Learning to improve customer experiences, fraud-resilient automated document processing, and data quality and observability."

### Question Answering Chain

In [ ]:
# Ingest PDF files
from langchain.document_loaders import PyPDFLoader

# Load GOOG's 10K annual report (92 pages).
url = "https://abc.xyz/investor/static/pdf/20230203_alphabet_10K.pdf"
loader = PyPDFLoader(url)
documents = loader.load()

In [ ]:

# split the documents into chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=0)
docs = text_splitter.split_documents(documents)
print(f"# of documents = {len(docs)}")

# of documents = 263


In [ ]:
# select embedding engine - we use Vertex PaLM Embeddings API
embeddings=

NameError: name 'embeddings' is not defined

In [ ]:
# Store docs in local vectorstore as index
# it may take a while since API is rate limited
from langchain.vectorstores import Chroma

db = Chroma.from_documents(docs, embeddings)

NameError: name 'embeddings' is not defined

In [ ]:
# Expose index to the retriever
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 2})

In [ ]:
# Create chain to answer questions
from langchain.chains import RetrievalQA

# Uses LLM to synthesize results from the search index.
# We use Vertex PaLM Text API for LLM
qa = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=retriever, return_source_documents=True
)

In [ ]:
query = "What was Alphabet's net income in 2022?"
result = qa({"query": query})
print(result)

In [ ]:
query = "How much office space reduction took place in 2023?"
result = qa({"query": query})
print(result)